# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cabo san lucas,22.8909,-109.9124,26.45,68,67,1.79,MX,1709682576
1,1,kerikeri,-35.2268,173.9474,21.62,68,62,0.45,NZ,1709683030
2,2,avarua,-21.2078,-159.7750,29.03,70,20,5.66,CK,1709682909
3,3,jalna,19.8333,75.8833,21.88,23,59,3.15,IN,1709683334
4,4,isafjordur,66.0755,-23.1240,1.88,79,100,2.91,IS,1709682911


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot_a_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    scale = 0.5,
    frame_widith = 700,
    frame_height = 500
)

# Display the map
plot_a_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
conditions_df = city_data_df

# Drop any rows with null values
conditions_df = conditions_df.dropna()

#max temperature lower than 27 degrees but higher than 21
conditions_df = conditions_df.loc[conditions_df["Max Temp"] < 27]
conditions_df = conditions_df.loc[conditions_df["Max Temp"] > 21]

#wind speed less than 4.5 m/s
conditions_df = conditions_df.loc[conditions_df["Wind Speed"] < 4.5]

# zero cloudiness
conditions_df = conditions_df.loc[conditions_df["Cloudiness"] == 0]

# Display sample data
conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,blackmans bay,-43.0167,147.3167,22.07,52,0,0.45,AU,1709682906
102,102,chibuto,-24.6867,33.5306,25.58,67,0,4.22,MZ,1709683361
127,127,santiago juxtlahuaca,17.3333,-98.0167,22.70,42,0,3.37,MX,1709683366
168,168,new norfolk,-42.7826,147.0587,22.73,36,0,2.13,AU,1709682942
200,200,galvez,-32.0293,-61.2210,21.62,55,0,1.91,AR,1709683379
363,363,duba,27.3513,35.6901,21.48,51,0,2.02,SA,1709683416
550,550,calingasta,-31.3308,-69.4078,24.47,44,0,3.68,AR,1709683500
555,555,gladewater,32.5365,-94.9427,26.49,38,0,0.00,US,1709683501


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = conditions_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
34,blackmans bay,AU,-43.0167,147.3167,52,
102,chibuto,MZ,-24.6867,33.5306,67,
127,santiago juxtlahuaca,MX,17.3333,-98.0167,42,
168,new norfolk,AU,-42.7826,147.0587,36,
200,galvez,AR,-32.0293,-61.2210,55,
363,duba,SA,27.3513,35.6901,51,
550,calingasta,AR,-31.3308,-69.4078,44,
555,gladewater,US,32.5365,-94.9427,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "apiKey" : geoapify_key, 
    "categories" : "accommodation.hotel",
    # "limit" : 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {lon}, {lat}, {radius}"
    params["bias"] = f"proximity: {lon}, {lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: No hotel found
chibuto - nearest hotel: No hotel found
santiago juxtlahuaca - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
galvez - nearest hotel: No hotel found
duba - nearest hotel: No hotel found
calingasta - nearest hotel: No hotel found
gladewater - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
34,blackmans bay,AU,-43.0167,147.3167,52,No hotel found
102,chibuto,MZ,-24.6867,33.5306,67,No hotel found
127,santiago juxtlahuaca,MX,17.3333,-98.0167,42,No hotel found
168,new norfolk,AU,-42.7826,147.0587,36,No hotel found
200,galvez,AR,-32.0293,-61.2210,55,No hotel found
363,duba,SA,27.3513,35.6901,51,No hotel found
550,calingasta,AR,-31.3308,-69.4078,44,No hotel found
555,gladewater,US,32.5365,-94.9427,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
plot_a_map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color = "City",
    size="Humidity",
    frame_widith = 700,
    frame_height = 500
)

# Display the map
plot_a_map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)